## Imports

In [4]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [5]:
df = pd.read_csv('WPB_map_template_popup_pin - Sheet1.csv')

## Image URL collector

In [6]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [7]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

In [8]:
len(links)

11

## Article link formatter

In [9]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2022/11...
1     <a href="https://therealdeal.com/miami/2022/08...
2     <a href="https://therealdeal.com/miami/2022/08...
3     <a href="https://therealdeal.com/miami/2022/07...
4     <a href="https://therealdeal.com/miami/2022/07...
5     <a href="https://therealdeal.com/miami/2022/07...
6     <a href="https://therealdeal.com/miami/2022/07...
7     <a href="https://therealdeal.com/miami/2022/06...
8     <a href="https://therealdeal.com/miami/2022/05...
9     <a href="https://therealdeal.com/miami/2022/04...
10    <a href="https://therealdeal.com/miami/2021/12...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [10]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [11]:
df['geo_address'] = df['full_address'] + ' West Palm Beach, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [12]:
df.columns

Index(['full_address', 'developers', 'land_purchase', 'loan_amount',
       'condos_or_apartments_or_townhomes', 'num_of_units', 'status',
       'description', 'story_link', 'image_links', 'short_description',
       'remaining_desc', 'description_link', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
df

,full_address,developers,land_purchase,loan_amount,condos_or_apartments_or_townhomes,num_of_units,status,description,story_link,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,2121 North Flagler Drive,Jeff Greene,20000000.0,NaN,Condos,182,Proposed,Billionaire Jeff Greene wants to build two 30-...,https://therealdeal.com/miami/2022/11/04/jeff-...,https://s11.therealdeal.com/trd/m/up/2022/11/M...,Billionaire Jeff,Greene wants to build two 30-story condominum...,"<a href=""https://therealdeal.com/miami/2022/11...","2121 North Flagler Drive West Palm Beach, FL","(2121 N Flagler Dr, West Palm Beach, FL 33407,...","(26.733614, -80.051267, 0.0)",26.733614,-80.051267,0.0
1,550 North Rosemary Avenue,Estate Companies,15800000.0,NaN,Apartments,321,Proposed,South Miami-based Estate Companies proposed a ...,https://therealdeal.com/miami/2022/08/23/estat...,https://s13.therealdeal.com/trd/m/up/2022/08/E...,South Miami-based,Estate Companies proposed a 321-apartment bui...,"<a href=""https://therealdeal.com/miami/2022/08...","550 North Rosemary Avenue West Palm Beach, FL","(550 N Rosemary Ave, West Palm Beach, FL 33401...","(26.7176095, -80.0567877, 0.0)",26.717609,-80.056788,0.0
2,418-444 Nottingham Boulevard,NDT Development,3000000.0,NaN,Townhomes,16,Planned,Ned Grace and Damien Barr's West Palm Beach-ba...,https://therealdeal.com/miami/2022/08/19/nora-...,https://s13.therealdeal.com/trd/m/up/2022/08/m...,Ned Grace and Damien,Barr's West Palm Beach-based NDT Development ...,"<a href=""https://therealdeal.com/miami/2022/08...","418-444 Nottingham Boulevard West Palm Beach, FL","(418 Nottingham Blvd, West Palm Beach, FL 3340...","(26.6741194, -80.05560969999999, 0.0)",26.674119,-80.055610,0.0
3,1919 North Flagler Drive,Savanna,NaN,NaN,"Condos, apartments","315 condos, 170 apts",Planned,"New York City-based Savanna plans Olara, a res...",https://therealdeal.com/miami/2022/07/22/olara...,https://s11.therealdeal.com/trd/m/up/2022/07/M...,New York City-based,"Savanna plans Olara, a residential complex wi...","<a href=""https://therealdeal.com/miami/2022/07...","1919 North Flagler Drive West Palm Beach, FL","(1919 N Flagler Dr, West Palm Beach, FL 33407,...","(26.7304458, -80.05021789999999, 0.0)",26.730446,-80.050218,0.0
4,1355 South Flagler Drive,Hines and Frisbie Group,41500000.0,33500000.0,Condos,89,Planned,Houston-based Hines and Palm Beach-based Frisb...,https://therealdeal.com/miami/2022/07/18/devel...,https://s11.therealdeal.com/trd/m/up/2022/07/m...,Houston-based Hines,and Palm Beach-based Frisbie Group plan a pai...,"<a href=""https://therealdeal.com/miami/2022/07...","1355 South Flagler Drive West Palm Beach, FL","(1355 S Flagler Dr, West Palm Beach, FL 33401,...","(26.7012994, -80.0510372, 0.0)",26.701299,-80.051037,0.0
5,2750 Golf Club Cir,Kolter Group,36900000.0,NaN,Townhomes,707,Proposed,"Delray Beach-based Kolter Group and Arlington,...",https://therealdeal.com/miami/2022/07/15/kolte...,https://s12.therealdeal.com/trd/m/up/2022/07/M...,Delray Beach-based,"Kolter Group and Arlington, Texas-based home ...","<a href=""https://therealdeal.com/miami/2022/07...","2750 Golf Club Cir West Palm Beach, FL","(2750 Golf Club Cir, West Palm Beach, FL 33417...","(26.7116599, -80.12094180000001, 0.0)",26.711660,-80.120942,0.0
6,740 and 840 North Dixie Highway,Tortoise Properties,18800000.0,88500000.0,Apartments,264,Planned,West Palm Beach-based Tortoise plan a pair of ...,https://therealdeal.com/miami/2022/07/06/torto...,https://s12.therealdeal.com/trd/m/up/2022/07/M...,West Palm Beach-,based Tortoise plan a pair of eight-story buil...,"<a href=""https://therealdeal.com/miami/2022/07...",740 and 840 North Dixie Highway West Palm Beac...,"(840 N Dixie Hwy, West Palm Beach, FL 33401, U...","(26.7202339, -80.0531847, 0.0)",26.720234,-80.053185,0.0
7,415 South Dixie Highway,13th Floor Investments and Wexford Real Estate...,26100000.0,NaN,Apartments,372,Proposed,Coconut Grove-based 13th Floor and

## Correction section

In [14]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [15]:
df.columns

Index(['full_address', 'developers', 'land_purchase', 'loan_amount',
       'condos_or_apartments_or_townhomes', 'num_of_units', 'status',
       'description', 'story_link', 'image_links', 'short_description',
       'remaining_desc', 'description_link', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

In [16]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developer) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [17]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [18]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/11/04/jeff-greene-plans-waterfront-west-palm-beach-condo-towers/" target="_blank" rel="noopener noreferrer">Billionaire Jeff</a> Greene wants to build two 30-story condominum towers totaling 321 units. Entities linked to Greene bought the property for $20 million in 2015. Greene retained Basel, Switzerland-based architecture firm Herzog & de Meuron for the project. Prices for the units have not yet been set.'

In [19]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'West Palm Beach Resi Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [20]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [21]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [22]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/west_palm_beach
